In [ ]:
import os
os.environ['NGROK_TOKEN'] = 'あなたのNgrokトークン'  # コピーしたトークンをここに貼り付け

In [ ]:
import os
import torch
from transformers import pipeline
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from contextlib import asynccontextmanager

# モデル名を変更
MODEL_NAME = "rinna/japanese-gpt-neox-3.6b-instruction-sft"  # よりアクセスしやすいモデルに変更

# モデルのグローバル変数
model = None

def load_model():
    """推論用のLLMモデルを読み込む"""
    global model
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"使用デバイス: {device}")
        pipe = pipeline(
            "text-generation",
            model=MODEL_NAME,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=device
        )
        print(f"モデル '{MODEL_NAME}' の読み込みに成功しました")
        model = pipe
        return pipe
    except Exception as e:
        print(f"モデル '{MODEL_NAME}' の読み込みに失敗: {e}")
        return None

@asynccontextmanager
async def lifespan(app: FastAPI):
    """アプリケーションのライフサイクル管理"""
    # 起動時の処理
    load_model()
    if model is None:
        print("警告: 起動時にモデルの初期化に失敗しました")
    else:
        print("起動時にモデルの初期化が完了しました。")
    yield
    # シャットダウン時の処理
    print("アプリケーションをシャットダウンしています...")

# FastAPIアプリケーション定義
app = FastAPI(
    title="LLM推論API",
    description="日本語LLMを使用したテキスト生成のためのAPI",
    version="1.0.0",
    lifespan=lifespan
)

# CORSミドルウェアを追加
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# データモデル定義
class GenerationRequest(BaseModel):
    prompt: str
    max_new_tokens: Optional[int] = 512
    do_sample: Optional[bool] = True
    temperature: Optional[float] = 0.7
    top_p: Optional[float] = 0.9

class GenerationResponse(BaseModel):
    generated_text: str

@app.get("/")
async def root():
    """基本的なAPIチェック用のルートエンドポイント"""
    return {"status": "ok", "message": "LLM API is running"}

@app.get("/health")
async def health_check():
    """ヘルスチェックエンドポイント"""
    global model
    if model is None:
        return {"status": "error", "message": "No model loaded"}
    return {"status": "ok", "model": MODEL_NAME}

@app.post("/generate", response_model=GenerationResponse)
async def generate(request: GenerationRequest):
    """テキスト生成エンドポイント"""
    global model

    if model is None:
        print("モデルが読み込まれていません。読み込みを試みます...")
        load_model()
        if model is None:
            raise HTTPException(status_code=503, detail="モデルが利用できません。")

    try:
        print(f"リクエストを受信: prompt={request.prompt[:100]}...")
        outputs = model(
            request.prompt,
            max_new_tokens=request.max_new_tokens,
            do_sample=request.do_sample,
            temperature=request.temperature,
            top_p=request.top_p,
        )
        
        generated_text = outputs[0]["generated_text"]
        if request.prompt in generated_text:
            generated_text = generated_text[len(request.prompt):].strip()
        
        return GenerationResponse(generated_text=generated_text)

    except Exception as e:
        print(f"応答生成中にエラーが発生しました: {e}")
        raise HTTPException(status_code=500, detail=f"応答の生成中にエラーが発生しました: {str(e)}")

# ngrokでAPIサーバーを実行
def run_with_ngrok(port=8000):
    """ngrokでFastAPIアプリを実行"""
    nest_asyncio.apply()
    
    ngrok_token = os.environ.get("NGROK_TOKEN")
    if not ngrok_token:
        print("Ngrok認証トークンが設定されていません。")
        print("以下のコマンドを実行してトークンを設定してください：")
        print("os.environ['NGROK_TOKEN'] = 'あなたの認証トークン'")
        return
    
    try:
        ngrok.set_auth_token(ngrok_token)
        public_url = ngrok.connect(port).public_url
        print(f"ngrok URL: {public_url}")
        uvicorn.run(app, host="0.0.0.0", port=port)
    except Exception as e:
        print(f"ngrok起動中にエラーが発生しました: {e}")

if __name__ == "__main__":
    run_with_ngrok()